In [1]:

import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import sqlite3
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:

conn = sqlite3.connect('./data/cars.db')
sql_query = '''

      SELECT
        coalesce(vin, title + ' ' + dealership_name + ' - ' + DATE(scraped_date, 'start_month')) as vin
        , transmission
        , row_number() OVER (PARTITION BY vin, DATE(scraped_date, 'start of month')  ORDER BY scraped_date ASC) AS filter_row
      FROM
        inventory
      WHERE
        scraped_date >= 2022-03-01 and scraped_date <= 2022-08-29

'''
result = pd.read_sql_query(sql_query, conn)
result

,vin,transmission,filter_row


In [6]:

conn = sqlite3.connect('./data/cars.db')
sql_query = '''

      SELECT
          *
      FROM
        inventory
      WHERE
       make is not null

'''
result = pd.read_sql_query(sql_query, conn)
result

,title,year,make,model_trim,vehicle_type,model,trim,vehicle_mileage,price,exterior_color,interior_color,transmission,engine,drivetrain,vin,dealership_name,dealership_address,dealership_zipcode,dealership_city,dealership_state,inventory_url,scraped_date
0,2009 Toyota Camry LE Sedan,2009,Toyota,Camry LE,Sedan,None,None,124351.0,8495.0,Magnetic Gray Metallic,Ash,Automatic,2.4L I4 158hp 161ft. lbs.,None,4T4BE46K89R114118,Bostonyan Auto Group,119 Worcester St,01760,Natick,MA,https://www.bostonyanautogroup.com/view-inventory,2022-03-14 17:34:28.561084
1,2011 BMW X3 xDrive28i SUV,2011,BMW,X3 xDrive28i,SUV,None,None,98952.0,13995.0,Jet Black,Black Nevada Leather,Automatic,3.0L I6 240hp 221ft. lbs.,None,5UXWX5C51BL715699,Bostonyan Auto Group,119 Worcester St,01760,Natick,MA,https://www.bostonyanautogroup.com/view-inventory,2022-03-14 17:34:28.561084
2,2011 Mercedes-Benz ML 350 4MATIC SUV,2011,Mercedes-Benz,ML 350 4MATIC,SUV,None,None,98922.0,14995.0,Obsidian Black Metallic,Black Leather,Automatic,3.5L V6 268hp 258ft. lbs.,None,4JGBB8GB4BA723044,Bostonyan Auto Group,119 Worcester St,01760,Natick,MA,https://www.bostonyanautogroup.com/view-inventory,2022-03-14 17:34:28.561084
3,2012 Audi A6 3.0T quattro Prestige Sedan,2012,Audi,A6 3.0T quattro Prestige,Sedan,None,None,98933.0,17995.0,White,Brown,Automatic,3.0L Supercharged V6 310hp 325ft. lbs.,None,WAUHGAFC4CN137257,Bostonyan Auto Group,119 Worcester St,01760,Natick,MA,https://www.bostonyanautogroup.com/view-inventory,2022-03-14 17:34:28.561084
4,2014 BMW 535i xDrive Sedan,2014,BMW,535i xDrive,Sedan,None,None,72940.0,23995.0,Silver,Tan,Automatic,3.0L Turbo I6 300hp 300ft. lbs.,None,WBA5B3C5XED537237,Bostonyan Auto Group,119 Worcester St,01760,Natick,MA,https://www.bostonyanautogroup.com/view-inventory,2022-03-14 17:34:28.561084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9330,2007 Mitsubishi Outlander LS,2007,Mitsubishi,Outlander LS,Sport Utility,Outlander,LS,93749.0,4998.0,Silver,,Automatic,6 Cyl - 3.0 L,AWD,JA4MT31X27Z013225,Irwin Automotive Group,59 Bisson Avenue,03246,Laconia,NH,https://www.irwinzone.com/searchused.aspx?pn=50,2022-08-21 21:11:40.399290
9331,2007 Jeep Grand Cherokee Laredo,2007,Jeep,Grand Cherokee Laredo,Sport Utility,Grand Cherokee,Laredo,170210.0,2698.0,Tan,,Automatic,6 Cyl - 3.7 L,4WD,1J8GR48K27C675433,Irwin Automotive Group,59 Bisson Avenue,03246,Laconia,NH,https://www.irwinzone.com/searchused.aspx?pn=50,2022-08-21 21:11:40.399290
9332,2005 Lexus ES 330,2005,Lexus,ES 330,4dr Car,ES 330,,175668.0,3998.0,Gold,,Automatic,6 Cyl - 3.3 L,FWD,JTHBA30GX55067864,Irwin Automotive Group,59 Bisson Avenue,03246,Laconia,NH,https://www.irwinzone.com/searchused.aspx?pn=50,2022-08-21 21:11:40.399290
9333,2004 Toyota Camry,2004,Toyota,Camry Base,4dr Car,Camry,Base,168436.0,4898.0,Blue,,,4 Cyl - 2.4 L,FWD,4T1BE32K94U288704,Irwin Automotive Group,59 Bisson Avenue,03246,Laconia,NH,https://www.irwinzone.com/searchused.aspx?pn=50,2022-08-21 21:11:40.399290


In [4]:
# ## Targeted drop
# TABLE_NAME = 'inventory'

# #Connecting to sqlite
# conn = sqlite3.connect('./data/cars.db')

# #Creating a cursor object using the cursor() method
# cursor = conn.cursor()

# #Doping EMPLOYEE table if already exists
# cursor.execute(f"DELETE FROM {TABLE_NAME} WHERE make is null")

# #Commit your changes in the database
# conn.commit()

# #Closing the connection
# conn.close()